![alt text](image.png)

### 思路
- 先初始化 形成一个相同的大小的矩阵
- 然后遍历原矩阵，特殊的是边界
  - 边界怎么办呢 padding = 0? 
    - 好思路，
    - 原来的矩阵大小就变n+2 * n+2
    - 同时对应关系是 行列各加一
    - 那么就有从1，1start然后直到n+1,n+1
  - 正常的八个怎么统计活细胞
    - 活细胞为1 
    - 那么就是说相加大于3 -》0
    - 相加等于 3 -》1
    - 相加等于 2 and 原来是1 -》 1
    - 相加小于2 -》0
   

In [9]:
from typing import List

class Solution:
    def gameOfLife(self, board: List[List[int]]) -> None:
        rows = len(board)
        cols = len(board[0])
        # 新建二维矩阵不然会报错
        pad_board = [[ _ for _ in range(cols + 2)] for _ in range(rows + 2)]
        # 添加padding
        for i in range(rows+2):
            for j in range(cols+2):
                if (i == 0 or j == 0 or i == rows + 1 or j == cols +1):
                    pad_board[i][j] = 0
                else:
                    pad_board[i][j] = board[i-1][j-1]
        # 进行判断
        for i in range(1,rows+1):
            for j in range(1,cols+1):
                temp = pad_board[i-1][j-1] + pad_board[i-1][j] + pad_board[i-1][j+1] + pad_board[i][j-1] + pad_board[i][j+1] + pad_board[i+1][j-1] + pad_board[i+1][j] + pad_board[i+1][j+1]
                if temp >3:
                    board[i-1][j-1] = 0
                elif temp == 3:
                    board[i-1][j-1] = 1
                elif temp == 2 and board[i-1][j-1] == 1:
                    board[i-1][j-1] = 1
                elif temp < 2:
                    board[i-1][j-1] = 0
        # 对照padding
        return board
    
a = Solution()
a.gameOfLife([[0,1,0],[0,0,1],[1,1,1],[0,0,0]])
            

[[0, 0, 0], [1, 0, 1], [0, 1, 1], [0, 1, 0]]

举个例子，如果细胞之前的状态是 0，但是在更新之后变成了 1，我们就可以给它定义一个复合状态 2。这样我们看到 2，既能知道目前这个细胞是活的，还能知道它之前是死的。



In [6]:
import numpy as np
from scipy import signal
from typing import List

class Solution:
    def gameOfLife(self, board: List[List[int]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        b = np.array(board)
        c = np.array([[1,1,1],[1,0,1],[1,1,1]])  # 卷积核
        x = signal.convolve(b, c, mode='same')  # 进行卷积
        
        # 更新细胞状态
        b[(x < 2) | (x > 3)] = 0  # 少于2个活细胞或多于3个活细胞，细胞死亡
        b[(x == 3)] = 1  # 恰好3个活细胞，细胞生存
        
        # 对原来是活细胞但卷积值是2的部分保持不变
        b[(x == 2) & (b == 1)] = 1
        
        # 将 NumPy 数组转换回列表并更新原始板
        board[:] = b.tolist()
    
a = Solution()
initial_board = [[0,1,0],[0,0,1],[1,1,1],[0,0,0]]
a.gameOfLife(initial_board)
print(initial_board)


[[0, 0, 0], [1, 0, 1], [0, 1, 1], [0, 1, 0]]


In [8]:
# 不用新建一个padding来解决边界的问题： 引入了复合状态
class Solution:
    def gameOfLife(self, board: List[List[int]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """

        neighbors = [(1,0), (1,-1), (0,-1), (-1,-1), (-1,0), (-1,1), (0,1), (1,1)]

        rows = len(board)
        cols = len(board[0])

        # 遍历面板每一个格子里的细胞
        for row in range(rows):
            for col in range(cols):

                # 对于每一个细胞统计其八个相邻位置里的活细胞数量
                live_neighbors = 0
                for neighbor in neighbors:

                    # 相邻位置的坐标
                    r = (row + neighbor[0])
                    c = (col + neighbor[1])

                    # 查看相邻的细胞是否是活细胞
                    if (r < rows and r >= 0) and (c < cols and c >= 0) and abs(board[r][c]) == 1:
                        live_neighbors += 1

                # 规则 1 或规则 3 
                if board[row][col] == 1 and (live_neighbors < 2 or live_neighbors > 3):
                    # -1 代表这个细胞过去是活的现在死了
                    board[row][col] = -1
                # 规则 4
                if board[row][col] == 0 and live_neighbors == 3:
                    # 2 代表这个细胞过去是死的现在活了
                    board[row][col] = 2

        # 遍历 board 得到一次更新后的状态
        for row in range(rows):
            for col in range(cols):
                if board[row][col] > 0:
                    board[row][col] = 1
                else:
                    board[row][col] = 0
        return board
a = Solution()
a.gameOfLife([[0,1,0],[0,0,1],[1,1,1],[0,0,0]])


[[0, 0, 0], [1, 0, 1], [0, 1, 1], [0, 1, 0]]